In [2]:
import horton
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [27]:
def eval_vanishing_site(fromZs, site, lval, grid):
    horton.log.set_level(0)
    mol = horton.IOData.from_file('../../test/benzene.xyz')
    mol.numbers = np.array(fromZs, dtype=int)
    toZs = mol.numbers.copy()
    toZs[site] = 0
    mol.pseudo_numbers = np.array(toZs, dtype=float) * lval + (1.-lval)*np.array(fromZs, dtype=float)
    
    obasis = horton.get_gobasis(mol.coordinates, mol.numbers, 'STO-3G')
    olp = obasis.compute_overlap()
    kin = obasis.compute_kinetic()
    na = obasis.compute_nuclear_attraction(mol.coordinates, mol.pseudo_numbers)
    er = obasis.compute_electron_repulsion()
    
    orb_alpha = horton.Orbitals(obasis.nbasis)
    orb_beta = horton.Orbitals(obasis.nbasis)
    
    one = kin + na
    horton.guess_core_hamiltonian(olp, one, orb_alpha, orb_beta)
    
    external = {'nn': horton.compute_nucnuc(mol.coordinates, mol.pseudo_numbers)}
    
    terms = [horton.UTwoIndexTerm(kin, 'kin'), horton.UDirectTerm(er, 'hartree'), horton.UExchangeTerm(er, 'x_hf'), horton.UTwoIndexTerm(na, 'ne')]
    ham = horton.UEffHam(terms, external)
    occ_model = horton.AufbauOccModel(21, 21)
    occ_model.assign(orb_alpha, orb_beta)
    dm_alpha = orb_alpha.to_dm()
    dm_beta = orb_beta.to_dm()
    scf_solver = horton.EDIIS2SCFSolver(1e-5, maxiter=500)
    scf_solver(ham, olp, occ_model, dm_alpha, dm_beta)
    
    fock_alpha = np.zeros(olp.shape)
    fock_beta = np.zeros(olp.shape)
    ham.reset(dm_alpha, dm_beta)
    energy = ham.compute_energy()    
    
    # integration grid
    rho_alpha = obasis.compute_grid_density_dm(dm_alpha, grid.points)
    rho_beta = obasis.compute_grid_density_dm(dm_beta, grid.points)
    rho_full = rho_alpha + rho_beta
    
    return energy, rho_full

def get_grid():
    mol = horton.IOData.from_file('../../test/benzene.xyz')
    grid = horton.BeckeMolGrid(mol.coordinates, mol.numbers, mol.pseudo_numbers, random_rotate=False)
    return grid
def get_coordinates():
    mol = horton.IOData.from_file('../../test/benzene.xyz')
    return mol.coordinates

In [58]:
def get_site_energy(site):
    lvals = []
    energies = []
    rhos = []
    grid = get_grid()
    Zs = [6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1]
    for lval in np.linspace(0, 1, 10):
        print (lval)
        lvals.append(lval)
        e, rho = eval_vanishing_site(Zs, site, lval, grid)
        rhos.append(rho)
        energies.append(e)
    int_rho = np.trapz(rhos, lvals, axis=0)
    ds = np.linalg.norm((get_coordinates()[site] - grid.points), axis=1)
    
    deltaE = grid.integrate(int_rho/ds)*(-Zs[site])
    Zstarget = [_ for _ in Zs]
    Zstarget[site] = 0
    deltaNN = horton.compute_nucnuc(get_coordinates(), np.array(Zs, dtype=float))-horton.compute_nucnuc(get_coordinates(), np.array(Zstarget, dtype=float))
    return deltaE, deltaNN/2
#plt.plot(lvals, energies)

In [59]:
(get_site_energy(0) , get_site_energy(6))

0.0
0.1111111111111111
0.2222222222222222
0.3333333333333333
0.4444444444444444
0.5555555555555556
0.6666666666666666
0.7777777777777777
0.8888888888888888
1.0
0.0
0.1111111111111111
0.2222222222222222
0.3333333333333333
0.4444444444444444
0.5555555555555556
0.6666666666666666
0.7777777777777777
0.8888888888888888
1.0


((-101.69383631306903, 29.164676702870892),
 (-10.150147765831752, 4.671864381591959))